In [1]:
# Libraries
import pandas as pd
from functools import reduce
from datetime import datetime, timedelta
import yfinance as yf

import urllib
import json

In [38]:
### y finance price data download

# Set the dates
start = datetime(2018,2,1)
end = datetime(2023,6,30)

# Set the ticker
ticker_yf = 'TSLA'

# Get the data
price_df = yf.download(ticker_yf, start, end)
price_df = price_df[['Adj Close', 'Volume']]

[*********************100%***********************]  1 of 1 completed


In [39]:
ticker_eod = ticker_yf + '.US'
api_key = ''

url = 'https://eodhistoricaldata.com/api/fundamentals/' + ticker_eod + '?api_token=' +api_key

# demo
# url = 'https://eodhistoricaldata.com/api/fundamentals/AAPL.US?api_token=demo'

response = urllib.request.urlopen(url)
data = json.loads(response.read())

In [40]:
# dates for for loop
quarter_list = list(data['Financials']['Balance_Sheet']['quarterly'].keys())[:40]

# create empty frame
fundamental_df = pd.DataFrame(index= quarter_list)

In [41]:
date_list = []
shares_list = []
for i in range(40):
  # date_list.append(data['outstandingShares']['quarterly'][str(i)]['dateFormatted'])
  shares_list.append(data['outstandingShares']['quarterly'][str(i)]['shares'])
fundamental_df['outstandingShares'] = shares_list

epsActual = []

for date in quarter_list:
 epsActual.append(data['Earnings']['History'][date]['epsActual'])
fundamental_df['epsActual'] = epsActual

# EV = Market Cap + Total Debt – Cash

# data['Financials']['Balance_Sheet']['quarterly']['2023-03-31'].keys()

totalAssets = []
totalStockholderEquity = []
shortLongTermDebtTotal = []
cashAndEquivalents = []

for date in quarter_list:
  totalAssets.append(data['Financials']['Balance_Sheet']['quarterly'][date]['totalAssets'])
  totalStockholderEquity.append(data['Financials']['Balance_Sheet']['quarterly'][date]['totalStockholderEquity'])
  shortLongTermDebtTotal.append(data['Financials']['Balance_Sheet']['quarterly'][date]['shortLongTermDebtTotal'])
  cashAndEquivalents.append(data['Financials']['Balance_Sheet']['quarterly'][date]['cashAndEquivalents'])

fundamental_df['totalAssets'] = totalAssets
fundamental_df['totalStockholderEquity'] = totalStockholderEquity
fundamental_df['shortLongTermDebtTotal'] = shortLongTermDebtTotal
fundamental_df['cashAndEquivalents'] = cashAndEquivalents


#########

# data['Financials']['Income_Statement']['quarterly']['2023-03-31'].keys()

netIncome = []
ebitda = []

for date in quarter_list:
  netIncome.append(data['Financials']['Income_Statement']['quarterly'][date]['netIncome'])
  ebitda.append(data['Financials']['Income_Statement']['quarterly'][date]['ebitda'])

fundamental_df['netIncome'] = netIncome
fundamental_df['ebitda'] = ebitda

In [42]:
# datetime index 로 만들기
fundamental_df.index = pd.to_datetime(fundamental_df.index)

# to_numeric
for column in fundamental_df.columns:
  fundamental_df[column] = pd.to_numeric(fundamental_df[column])

In [43]:
# roe, roa, ev/ebitda, p/e, debt/asset

In [44]:

# merge price and fundamentals --> then ffill
merged_df = pd.merge(price_df, fundamental_df, how='left', right_index=True, left_index=True)
merged_df = merged_df.fillna(method='ffill').dropna()

# making fundamentals

merged_df['roe'] = merged_df['netIncome'] / merged_df['totalStockholderEquity']
merged_df['roa'] = merged_df['netIncome'] / merged_df['totalAssets']
merged_df['debt/asset'] = merged_df['shortLongTermDebtTotal'] / merged_df['totalAssets']
merged_df['p/e'] = merged_df['Adj Close'] / merged_df['epsActual']
merged_df['ev'] = merged_df['Adj Close'] * merged_df['outstandingShares'] + merged_df['shortLongTermDebtTotal'] - merged_df['cashAndEquivalents']
merged_df['ev/ebitda'] = merged_df['ev'] / merged_df['ebitda']

# choosing only the necessary columns
final_df = merged_df[['Adj Close', 'Volume', 'roe', 'roa', 'debt/asset', 'ev/ebitda', 'p/e']]

In [45]:
final_df.head()

,Adj Close,Volume,roe,roa,debt/asset,ev/ebitda,p/e
Date,,,,,,,
2021-06-30,226.566666,56774700,0.046041,0.020709,0.201882,364.617904,472.013887
2021-07-01,225.973328,55903500,0.046041,0.020709,0.201882,363.656593,470.777766
2021-07-02,226.300003,81163500,0.046041,0.020709,0.201882,364.185864,471.458340
2021-07-06,219.860001,69853500,0.046041,0.020709,0.201882,353.751941,458.041668
2021-07-07,214.883331,56376000,0.046041,0.020709,0.201882,345.688872,447.673607


In [47]:
# data.keys()
# data['Valuation']
# data['Technicals']
# data['Financials']['Balance_Sheet']['quarterly']['2023-03-31'].keys()
# data['Financials']['Income_Statement']['quarterly']['2023-03-31'].keys()